In [ ]:
pip install opencv-python

In [ ]:
import cv2
import mediapipe as mp
import math
import numpy

In [ ]:
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume


devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

cap = cv2.VideoCapture(0)
mpDraw = mp.solutions.drawing_utils
drawing_styles = mp.solutions.drawing_styles
mpHands = mp.solutions.hands
hands = mpHands.Hands()

In [ ]:
while True:
    success, img = cap.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            lmList =[]
            for id, lm in enumerate(handLms.landmark):
                h,w,c = img.shape
                cx, cy = int(lm.x*w) , int(lm.y*h)
                lmList.append([id,cx, cy])


                #print(id, lm)
                #mpDraw.draw_landmark(img, handLms, mpHands.HAND_CONNECTIONS)
            if lmList:
                x1, y1 = lmList[4][1] , lmList[4][2]
                x2, y2 = lmList[8][1] , lmList[8][2]
                cv2.circle(img, (x1,y1), 5, (255,0,0))
                cv2.circle(img, (x2,y2), 5, (200,0,0))
                cv2.line(img, (x1,y1), (x2,y2), (255,128,0), 4)
                z1,z2 = (x1+x2)//2 , (y1+y2)//2
                length = math.hypot(x2-x1 , y2-y1)
                #print(length)
            volRange = volume.GetVolumeRange()
            minVol = volRange[0]
            maxVol = volRange[1]
            vol = numpy.interp(length, [50,100], [minVol, maxVol])
            volPer = numpy.interp(length, [50,100], [0,100])
            volBar = numpy.interp(length, [50,100], [400,150])
            volume.SetMasterVolumeLevel(vol, None)
            cv2.rectangle(img,(50,150), (85,400), (255,0,0), 3)
            cv2.rectangle(img,(50,int(volBar)), (85,400), (255, 128, 0), cv2.FILLED)


    cv2.imshow("image",img)
    cv2.waitKey(1)
